# Generate Contacts

This notebook fetches a spreadsheet where in not excluded worksheets the column 0 contains full name and column 2 contains phone number from Google Drive and then generates a CSV file compatible with Google Contacts. User should check sections related to handling of name, surname and phone number before using the generated data for production.

## Load Dependencies

In [0]:
!pip install --upgrade -q gspread

## Import Dependencies
This notebook depends on Google Colab, Google Spreadsheet, Oauth2client Helper, Regex modules to function.

In [0]:
from google.colab import auth, files
import gspread
from oauth2client.client import GoogleCredentials
import re

## Define Constants

In [0]:
header = 'Name,Given Name,Additional Name,Family Name,Yomi Name,Given Name Yomi,Additional Name Yomi,Family Name Yomi,Name Prefix,Name Suffix,Initials,Nickname,Short Name,Maiden Name,Birthday,Gender,Location,Billing Information,Directory Server,Mileage,Occupation,Hobby,Sensitivity,Priority,Subject,Notes,Language,Photo,Group Membership,Phone 1 - Type,Phone 1 - Value,Phone 2 - Type,Phone 2 - Value,Organization 1 - Type,Organization 1 - Name,Organization 1 - Yomi Name,Organization 1 - Title,Organization 1 - Department,Organization 1 - Symbol,Organization 1 - Location,Organization 1 - Job Description'
spreadsheet_name = ''
worksheets_to_exclude = [ '' ]
sample_phone_number = '900000000000'
file_name = 'contacts.csv'

## Authorize Runtime

In [0]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## Open Spreadsheet

In [0]:
spreadsheet = gc.open(spreadsheet_name)

## Define Row Builder
Google Contacts CSV format has many fields but this notebook is only concerned with name and number fields.

In [0]:
def build(name, surname, number):
    return '{} {},{},,{},,,,,,,,,,,,,,,,,,,,,,,,,* myContacts,Mobile,{},,,,,,,,,,'.format(
        name,
        surname,
        name,
        surname,
        number
    )

## Generate Rows

In [0]:
data = [ header ]

for worksheet in spreadsheet.worksheets():
    if worksheet.title not in worksheets_to_exclude:
        rows = worksheet.get_all_values()
        for row in rows[1:]:
            full_name = row[0].strip()
            surname = full_name.split(' ')[-1]
            name = ' '.join(full_name.split(' ')[0:-1])
            number = re.sub('[^0-9]', '', row[2].strip())
            if len(number) == len(sample_phone_number):
                data.append(build(name, surname, '+' + number))

print(len(data))

## Download File

In [0]:
with open(file_name, 'w') as f:
    for section in data:
        f.write(section)
        f.write('\n')

files.download(file_name)